# Assignment 1

This is an individual assignment, but you may collaborate with team mates on getting to know the data. 

## The dataset

Consider the cps-earnings dataset  at https://osf.io/g8p9j/ (Cross section. N=149 316 individuals)

Pick an occupation and filter data accordingly. You must all pick different occupations / occupation individually.  
Occupation codes are here: https://osf.io/57n9q/
You may merge occupations as you see fit (ie all tax/insurance specialists, etc). 
You can see some ideas working with this code here.

https://github.com/gabors-data-analysis/da_case_studies/tree/master/ch09-gender-age-earnings
https://github.com/gabors-data-analysis/da_case_studies/tree/master/ch10-gender-earnings-understand
## Tasks

Build four predictive models using linear regression for earnings per hour. 

1. Models: the target variable is earnings per hour, all others would be predictors.
2. Model 1 shall be the simplest, model 4 the more complex. It shall be OLS. You shall explain your choice of predictors.
3. Compare model performance of these models (a) RMSE in the full sample, (2) cross-validated RMSE and (c) BIC in the full sample. 
4. Discuss the relationship between model complexity and performance. You may use visual aids. 
5. You should submit your code in Github and 1 page report in pdf on Moodle. 

Work individually. But you may collaborate in your support group, check and comment (add issues) on each other code.

## Hints re Git and commit

Committing is a habit, and people may have different ways. 

Some people commit very frequently, others less so. 
We basically expect you to have a few commits, one per major parts of the exercise. The first commit will set up the the folder/file for A1. 
Then you can commit, say data work, descriptives, graphics, and regressions. And then, commit your edits. 
Make sure the commit text is short but meaningful: Good: "adding graphs", "calculate RMSE", "edit typos". Bad: "update"
Grading

This assignment is worth 20 points. 

* 5 points will be for Git use. 
* 10 points will be technical aspects the analysis 
* 5 points will be based on your report

### Setting up directories and import packages

In [31]:
### SETTING UP DIRECTORIES

# import packages
import pandas as pd
import os
import numpy as np
import sys
import warnings

from mizani.formatters import percent_format
from plotnine import *
from datetime import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
from mizani import transforms
from scipy.stats import norm
from IPython.core.display import HTML
from stargazer.stargazer import Stargazer
import statsmodels.nonparametric.kernel_regression as loess
import seaborn as sns
from mizani.formatters import percent_format
import regex as re

warnings.filterwarnings("ignore")

# set working directory for da_data_repo -- replace the
os.chdir('C:\\Users\\77774\\Documents\\GitHub\\Prediction_with_ML_for_Economists\\Assignment_1_cps_earnings_dataset')

# location folders
data_in = "C:\\Users\\77774\\Documents\\GitHub\\Prediction_with_ML_for_Economists\\Assignment_1_cps_earnings_dataset\\clean\\"
data_out = "C:\\Users\\77774\\Documents\\GitHub\\Prediction_with_ML_for_Economists\\Assignment_1_cps_earnings_dataset\\clean\\"
results = "C:\\Users\\77774\\Documents\\GitHub\\Prediction_with_ML_for_Economists\\Assignment_1_cps_earnings_dataset\\results\\"

In [52]:
# load dataset (as unicode, to avoid size and memor warnings)

df = pd.read_csv(
    data_in + "morg-2014-emp.csv",
    quotechar='"',
    delimiter=",",
    encoding="utf-8",
#    dtype = "unicode"
)

### Prepare data

In [53]:
# Keep only financial specialists

df.loc[
    ((df["occ2012"] >= 800) & (df["occ2012"] <= 950)), "sample"
] = 1
df.loc[df["sample"].isna(), "sample"] = 0

df = df.loc[
    (df["sample"] == 1), :
].reset_index(drop=True)

In [54]:
#create some variables

#female
df["female"] = (df.sex == 2).astype(int)

#w and lnw
df["w"] = df["earnwke"] / df["uhours"]
df["lnw"] = np.log(df["w"])

#age forms
df["agesq"] = np.power(df["age"], 2)
df["agecu"] = np.power(df["age"], 3)
df["agequ"] = np.power(df["age"], 4)

#married
df['married'] = (df.marital <= 4).astype(int)

#race
df["white"] = (df["race"] == 1).astype(int)
df["afram"] = (df["race"] == 2).astype(int)
df["asian"] = (df["race"] == 4).astype(int)
df["hisp"] = (df["ethnic"].notna()).astype(int)
df["othernonw"] = (
    (df["white"] == 0) & (df["afram"] == 0) & (df["asian"] == 0) & (df["hisp"] == 0)
).astype(int)

#nonUSborn
df["nonUSborn"] = (
    (df["prcitshp"] == "Foreign Born, US Cit By Naturalization")
    | (df["prcitshp"] == "Foreign Born, Not a US Citizen")
).astype(int)

#havechild = at least 1 child
df['havechild']=(df['chldpres']>=0).astype(int)

In [55]:
# transform grade92 to educ (which will be measured in years)
# create a list of our conditions
conditions = [
    (df['grade92'] == 31),
    (df['grade92'] == 32),
    (df['grade92'] == 33),
    (df['grade92'] == 34),
    (df['grade92'] == 35),
    (df['grade92'] == 36),
    (df['grade92'] == 37),
    (df['grade92'] == 38),
    (df['grade92'] == 39),
    (df['grade92'] == 40),
    (df['grade92'] == 41),
    (df['grade92'] == 42),
    (df['grade92'] == 43),
    (df['grade92'] == 44),
    (df['grade92'] == 45),
    (df['grade92'] == 46)
    ]

# create a list of the values we want to assign for each condition
values = [1, 4, 6, 8, 9, 10, 11, 12, 13, 14, 15, 15, 16, 18, 18, 20]

# create a new column and use np.select to assign values to it using our lists as arguments
df['educ'] = np.select(conditions, values)
df['educsq'] = np.power(df["educ"], 2)
df['educcu'] = np.power(df["educ"], 3)
df['educqu'] = np.power(df["educ"], 4)


In [68]:
# remove columns we don't need
df= df[
    [

'earnwke' ,
'uhours'   ,     
'age'      ,            
'occ2012'   ,        
'female'    ,     
'w'         ,   
'lnw'     ,     
'agesq'     ,   
'agecu'    ,      
'agequ'   ,       
'married'   ,     
'white'      ,    
'afram'       ,   
'asian'        ,  
'hisp'          , 
'othernonw'      ,
'nonUSborn'      ,
'havechild'      ,
'educ'           ,
'educsq'         ,
'educcu'        ,
'educqu'    
    ]
]

### Look at the data

In [71]:
#quick look
df.describe()

,earnwke,uhours,age,occ2012,female,w,lnw,agesq,agecu,agequ,...,afram,asian,hisp,othernonw,nonUSborn,havechild,educ,educsq,educcu,educqu
count,3404.000000,3404.000000,3404.000000,3404.000000,3404.000000,3404.000000,3404.000000,3404.000000,3404.000000,3.404000e+03,...,3404.000000,3404.000000,3404.000000,3404.000000,3404.000000,3404.0,3404.000000,3404.000000,3404.000000,3404.000000
mean,1252.053637,41.211222,42.066392,834.030552,0.603702,30.335379,3.267568,1902.685076,91328.103995,4.593613e+06,...,0.074912,0.080787,0.065805,0.014982,0.123090,1.0,15.814630,252.328731,4058.219448,65764.118390
std,685.573216,7.328230,11.538753,45.983876,0.489200,18.167321,0.626535,986.756564,67148.231997,4.266060e+06,...,0.263288,0.272548,0.247977,0.121500,0.328589,0.0,1.492269,45.770573,1085.429463,23280.293888
min,0.400000,1.000000,17.000000,800.000000,0.000000,0.010000,-4.605170,289.000000,4913.000000,8.352100e+04,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.000000,1.000000,1.000000,1.000000
25%,758.307500,40.000000,32.000000,800.000000,0.000000,19.000000,2.944439,1024.000000,32768.000000,1.048576e+06,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,15.000000,225.000000,3375.000000,50625.000000
50%,1057.690000,40.000000,42.000000,800.000000,1.000000,26.000000,3.258097,1764.000000,74088.000000,3.111696e+06,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,16.000000,256.000000,4096.000000,65536.000000
75%,1581.190000,40.000000,52.000000,850.000000,1.000000,38.451875,3.649407,2704.000000,140608.000000,7.311616e+06,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,16.000000,256.000000,4096.000000,65536.000000
max,2884.610000,92.000000,64.000000,950.000000,1.000000,500.000000,6.214608,4096.000000,262144.000000,1.677722e+07,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,20.000000,400.000000,8000.000000,160000.000000


In [72]:
# check we don't have missing variables
to_filter=df.isna().sum()
to_filter[to_filter>0].index

Index([], dtype='object')

### Setting up models

### Separate hold-out set

### RMSE in the full sample

### Cross-validated RMSE 

### BIC in the full sample

### Results

In [21]:
df

,lfsr94,hhid,lineno,intmonth,stfips,weight,earnwke,uhours,grade92,race,...,ownchild,chldpres,prcitshp,state,ind02,occ2012,class,unionmme,unioncov,sample
0,Employed-At Work,954001919079770,2,January,AL,3677.7816,680.0,40,43,1,...,0,0,"Native, Born In US",63,Non-depository credit and related activities (...,800,"Private, For Profit",No,No,1.0
1,Employed-At Work,507001320102921,1,January,AL,3296.7986,2000.0,50,44,1,...,1,1,"Native, Born In US",63,Real estate (531),800,"Private, For Profit",No,No,1.0
2,Employed-At Work,510915094903939,2,January,AL,3419.5125,1923.0,40,43,1,...,2,10,"Native, Born In US",63,"Banking and related activities (521, 52211,52219)",910,"Private, For Profit",No,No,1.0
3,Employed-At Work,210033043908195,5,January,AL,4563.6243,280.0,32,42,1,...,0,0,"Native, Born In US",63,"Recreational vehicle parks and camps, and room...",800,"Private, For Profit",No,No,1.0
4,Employed-At Work,981071966079500,2,January,AL,2416.8545,800.0,40,42,2,...,1,4,"Native, Born In US",63,"Savings institutions, including credit unions ...",910,"Private, For Profit",No,No,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3399,Employed-At Work,050510000344676,1,December,WI,3677.216,580.0,42,43,1,...,0,0,"Native, Born In US",3,** Construction (23),800,"Private, For Profit",No,No,1.0
3400,Employed-At Work,506902605991185,2,December,WI,403.3448,1538.46,48,43,1,...,0,0,"Native, Born In US",3,Nursing care facilities (6231),840,"Private, Nonprofit",No,No,1.0
3401,Employed-At Work,955161170004307,2,December,WI,3794.4402,1057.69,40,42,1,...,0,0,"Native, Born In US",3,Real estate (531),800,"Private, For Profit",No,No,1.0
3402,Employed-At Work,816460901300501,1,December,WY,275.7171,1923.07,40,43,1,...,1,3,"Native, Born In US",8,"Banking and related activities (521, 52211,52219)",910,"Private, For Profit",No,No,1.0


In [23]:
df["earnwke"].describe()

count        3404
unique        729
top       2884.61
freq          209
Name: earnwke, dtype: object

In [65]:
df["sex"].value_counts()

2    2055
1    1349
Name: sex, dtype: int64

In [67]:
for col in df:
    print(df["sex"].unique())

[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]


In [69]:
df.dtypes

earnwke      float64
uhours         int64
age            int64
occ2012        int64
female         int32
w            float64
lnw          float64
agesq          int64
agecu          int64
agequ          int64
married        int32
white          int32
afram          int32
asian          int32
hisp           int32
othernonw      int32
nonUSborn      int32
havechild      int32
educ           int32
educsq         int32
educcu         int32
educqu         int32
dtype: object

In [70]:
df.head()

,earnwke,uhours,age,occ2012,female,w,lnw,agesq,agecu,agequ,...,afram,asian,hisp,othernonw,nonUSborn,havechild,educ,educsq,educcu,educqu
0,680.0,40,27,800,0,17.000,2.833213,729,19683,531441,...,0,0,0,0,0,1,16,256,4096,65536
1,2000.0,50,48,800,0,40.000,3.688879,2304,110592,5308416,...,0,0,0,0,0,1,18,324,5832,104976
2,1923.0,40,47,910,0,48.075,3.872762,2209,103823,4879681,...,0,0,0,0,0,1,16,256,4096,65536
3,280.0,32,24,800,0,8.750,2.169054,576,13824,331776,...,0,0,0,0,0,1,15,225,3375,50625
4,800.0,40,42,910,1,20.000,2.995732,1764,74088,3111696,...,1,0,0,0,0,1,15,225,3375,50625


In [57]:
df.describe()

,hhid,lineno,weight,earnwke,uhours,grade92,race,ethnic,age,sex,...,afram,asian,hisp,othernonw,nonUSborn,havechild,educ,educsq,educcu,educqu
count,3.404000e+03,3404.000000,3404.000000,3404.000000,3404.000000,3404.000000,3404.000000,224.000000,3404.000000,3404.000000,...,3404.000000,3404.000000,3404.000000,3404.000000,3404.000000,3404.0,3404.000000,3404.000000,3404.000000,3404.000000
mean,4.389977e+14,1.561398,2352.193724,1252.053637,41.211222,42.428613,1.409224,2.892857,42.066392,1.603702,...,0.074912,0.080787,0.065805,0.014982,0.123090,1.0,15.814630,252.328731,4058.219448,65764.118390
std,3.231419e+14,0.765892,1255.320431,685.573216,7.328230,1.592634,1.134368,2.594776,11.538753,0.489200,...,0.263288,0.272548,0.247977,0.121500,0.328589,0.0,1.492269,45.770573,1085.429463,23280.293888
min,1.718648e+10,1.000000,149.658300,0.400000,1.000000,31.000000,1.000000,1.000000,17.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.000000,1.000000,1.000000,1.000000
25%,1.182361e+14,1.000000,1284.668025,758.307500,40.000000,42.000000,1.000000,1.000000,32.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,15.000000,225.000000,3375.000000,50625.000000
50%,4.043134e+14,1.000000,2679.500250,1057.690000,40.000000,43.000000,1.000000,1.000000,42.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,16.000000,256.000000,4096.000000,65536.000000
75%,7.209905e+14,2.000000,3303.234300,1581.190000,40.000000,43.000000,1.000000,5.000000,52.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,16.000000,256.000000,4096.000000,65536.000000
max,9.999404e+14,7.000000,7411.615800,2884.610000,92.000000,46.000000,21.000000,8.000000,64.000000,2.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,20.000000,400.000000,8000.000000,160000.000000
